# Evaluation Code

This notebook will be very __similar__ to the code I use to evaluate your results - it is provided for __your convenience__ so that you can use it to evaluate your preprocessing results at any time before your __final submission__.

Please note that the results here will __NOT__ be the same as my evaluation results.

Let's start with loading the required packages.

In [1]:
# import required package for data handling
import pandas as pd
import numpy as np

# import required packages for splitting data
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

# import required packages for evaluating models
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support

# import `logistic regression` model
from sklearn.linear_model import LogisticRegression

Next you should load __your__ data. In this case, I am using a sample dataset (`GroupX.csv`) which contains 6 predictors (`X1 - X6`) and two target variables (`Y1, Y2`).

Please make sure you change the data to your __OWN__ dataset when using this code.

__NOTE__:
1. Your dataset maybe very different from the sample dataset.
2. Please follow this structure when submitting your dataset.

In [2]:
## Reading in the data into one variable
data = pd.read_csv('Ready_DF_USE_ME', index_col=0)
data.head(2)

,C1,C2,C4,C7,C3',C5',C6',T3',T4',T5',S1',S2',S3',Y1,Y2
0,0.144811,0.397360,0.700156,-0.590043,0.942972,0.070221,1.133020,0.635348,0.182031,0.375422,-0.165963,0.276180,-0.104356,0,1
1,1.841533,-2.516611,-0.784155,-1.085427,-1.060477,-0.023130,-0.776623,0.210550,0.247332,-0.031227,-1.622714,0.972579,-0.128206,1,0


Checking your data types and make sure it follows the data dictionary would be an important step, you can do that using the `.dtypes` attribute.

__NOTE__: all __continuous__ faetures will be in `float64` data type, and all __categorical__ features will be in `int64` data type (given you already coded (per __suggest task \#6__ in the competition document) them).

In [3]:
data.dtypes

C1     float64
C2     float64
C4     float64
C7     float64
C3'    float64
C5'    float64
C6'    float64
T3'    float64
T4'    float64
T5'    float64
S1'    float64
S2'    float64
S3'    float64
Y1       int64
Y2       int64
dtype: object

In [13]:
data_drop_C6 = data.drop(["C6'"], axis=1)
data_drop_C6.head(2)

,C1,C2,C4,C7,C3',C5',T3',T4',T5',S1',S2',S3',Y1,Y2
0,0.144811,0.397360,0.700156,-0.590043,0.942972,0.070221,0.635348,0.182031,0.375422,-0.165963,0.276180,-0.104356,0,1
1,1.841533,-2.516611,-0.784155,-1.085427,-1.060477,-0.023130,0.210550,0.247332,-0.031227,-1.622714,0.972579,-0.128206,1,0


Now you need to specify your targets and predictors. __NOTE__ we have two targets here (`Y1, Y2`).

In [4]:
y1 = data.Y1
y2 = data.Y2

Check the shape of the data.

In [5]:
data.shape

(682, 15)

It is very possible that you will use different sets of the predictors for `Y1` and `Y2`. Now let's define them.

First, let's define predictors for `Y1` - which will be the first 5 features in `data`.

In [6]:
cols = list(data.columns)
# first 13 features 
cols[:-2]

['C1',
 'C2',
 'C4',
 'C7',
 "C3'",
 "C5'",
 "C6'",
 "T3'",
 "T4'",
 "T5'",
 "S1'",
 "S2'",
 "S3'"]

In [16]:
cols_no_C6 = list(data_drop_C6.columns)

cols_no_C6[:-2]

['C1',
 'C2',
 'C4',
 'C7',
 "C3'",
 "C5'",
 "T3'",
 "T4'",
 "T5'",
 "S1'",
 "S2'",
 "S3'"]

Use below code to select the first 5 features as predictors for `Y1`.

In [46]:
predictors_y1 = data[["C7","C3'","C5'","T3'","T4'","S1'","S2'","S3'"]]
predictors_y1.head()

,C7,C3',C5',T3',T4',S1',S2',S3'
0,-0.590043,0.942972,0.070221,0.635348,0.182031,-0.165963,0.276180,-0.104356
1,-1.085427,-1.060477,-0.023130,0.210550,0.247332,-1.622714,0.972579,-0.128206
2,-1.956202,-1.060477,-0.290469,0.655086,0.127568,-0.028577,-1.140452,0.104007
3,-1.860453,-1.060477,-0.344288,0.975481,-1.338731,2.316858,-0.658833,0.866386
4,1.268850,0.942972,-0.026516,-0.221809,-0.617840,-0.440742,0.602407,0.042702


Upon investigation of the data, we know we have __six__ features (`X1 - X6`) predicting `Y2`. Use similar code (as below) to select them.

In [18]:
predictors_y2 = data[['C2','C4','C7',"C3'","C5'","C6'","T3'","T5'","S1'","S2'","S3'"]]
predictors_y2.head()

,C2,C4,C7,C3',C5',C6',T3',T5',S1',S2',S3'
0,0.397360,0.700156,-0.590043,0.942972,0.070221,1.133020,0.635348,0.375422,-0.165963,0.276180,-0.104356
1,-2.516611,-0.784155,-1.085427,-1.060477,-0.023130,-0.776623,0.210550,-0.031227,-1.622714,0.972579,-0.128206
2,0.397360,0.407699,-1.956202,-1.060477,-0.290469,-0.776623,0.655086,1.351307,-0.028577,-1.140452,0.104007
3,0.397360,0.383494,-1.860453,-1.060477,-0.344288,-0.776623,0.975481,1.334986,2.316858,-0.658833,0.866386
4,0.397360,-1.537859,1.268850,0.942972,-0.026516,1.356978,-0.221809,-0.459604,-0.440742,0.602407,0.042702


Below is the key part of this notebook - which generates a `logistic regression` model to predict `Y1`/`Y2`.

The code works this way:

1. We generate two lists `f1_score_lst` and `auc_lst` to store f1_score and AUC from each of the `10` runs of the model;
2. Define model:
    1. We define a `LogisticRegression()` model;
    
    2. We split predictors (`predictors_y1`) and target `y1` to training (80%) and testing (20%);
    
    3. We fit the model `clf` to the training data, then use it to predict on the testing data;
    
    4. We also defined a `10-fold cross validation` to make sure our model do not overfit - see [here](https://scikit-learn.org/stable/modules/cross_validation.html) for more info;
    
    5. We append the f1_score and AUC of current model to the lists (`f1_score_lst` and `auc_lst`) we defined earlier.
  
3. Print out average f1_score and AUC for all 10 runs;
4. Print out average average accuracy from cross validation
5. Print out confusion matrix and classification report for the __last__ model.

__NOTE__: Step 3 provides the evaluation results we need; step 4 - 5 can be used to verify the results from step 3.

In [47]:
# lists for f1-score and AUC
f1_score_lst = []
auc_lst = []


#loop to calculate f1 and auc scores and present averages after 10 runs
for count in range (1,10):
    #Model building
    clf = LogisticRegression()
    X1_train, X1_test, y1_train, y1_test = train_test_split(predictors_y1, y1, test_size=0.3, random_state=123)
    clf.fit(X1_train, y1_train)

    y1_pred = clf.predict(X1_test)

    
    #10-fold cross validation
    kfold = model_selection.KFold(n_splits=10, random_state=7)
    scoring = 'accuracy'
    results = model_selection.cross_val_score(clf, X1_train, y1_train, cv=kfold, scoring=scoring)

    

    
    #calculate f1-score and AUC
    
    clf_roc_auc = roc_auc_score(y1_test, y1_pred)
    f1_score_lst.append(precision_recall_fscore_support(y1_test, y1_pred, average='weighted')[2])
    auc_lst.append(clf_roc_auc)


print('F1 {:.4f}; AUC {:.4f} '.format(np.mean(f1_score_lst),np.mean(auc_lst)))

#result=logit_model.fit()
confusion_matrix_y1 = confusion_matrix(y1_test, y1_pred)


#print(result.summary())
print('Accuracy of classifier on test set: {:.2f}'.format(clf.score(X1_test, y1_test)))

print("10-fold cross validation average accuracy of classifier: %.3f" % (results.mean()))

print('Confusion Matrix for Logistic Regression Classfier:')
print(confusion_matrix_y1)

print('Classification Report for Logistic Regression Classfier:')
print(classification_report(y1_test, y1_pred))


F1 0.5886; AUC 0.5881 
Accuracy of classifier on test set: 0.59
10-fold cross validation average accuracy of classifier: 0.602
Confusion Matrix for Logistic Regression Classfier:
[[52 47]
 [37 69]]
Classification Report for Logistic Regression Classfier:
             precision    recall  f1-score   support

          0       0.58      0.53      0.55        99
          1       0.59      0.65      0.62       106

avg / total       0.59      0.59      0.59       205



Below code are used to evaluate model toward `Y2`. It is very similar to the code above - key difference is that `Y2` is imbalanced - so I wrote some code (under `# Begin oversampling`) to deal with that.

In [10]:
# lists for f1-score and AUC
f1_score_lst = []
auc_lst = []


#loop to calculate f1 and auc scores and present averages after 10 runs
for count in range (1,10):
    #Model building
    clf1 = LogisticRegression()

    
    # Splitting data into testing and training
    X2_train, X2_test, y2_train, y2_test = train_test_split(predictors_y2, y2, test_size=0.2, random_state=123)
    
    # Begin oversampling
    oversample = pd.concat([X2_train,y2_train],axis=1)
    max_size = oversample['Y2'].value_counts().max()
    lst = [oversample]
    for class_index, group in oversample.groupby('Y2'):
        lst.append(group.sample(max_size-len(group), replace=True))
    X2_train = pd.concat(lst)
    y2_train=pd.DataFrame.copy(X2_train['Y2'])
    del X2_train['Y2']
    
    # fitting model on oversampled data
    clf1.fit(X2_train, y2_train)
    
    y2_pred = clf1.predict(X2_test)
    
    
    #10-fold cross validation
    kfold = model_selection.KFold(n_splits=10, random_state=123)
    scoring = 'accuracy'
    results = model_selection.cross_val_score(clf1, X2_train, y2_train, cv=kfold, scoring=scoring)
    
    #calculate f1-score and AUC
    
    clf1_roc_auc = roc_auc_score(y2_test, y2_pred)
    
    
    #calculate average f1-score and AUC
    f1_score_lst.append(precision_recall_fscore_support(y2_test, y2_pred, average='weighted')[2])
    auc_lst.append(clf1_roc_auc)
    
    
print('F1 {:.4f}; AUC {:.4f} '.format(np.mean(f1_score_lst),np.mean(auc_lst)))

confusion_matrix_y2 = confusion_matrix(y2_test, y2_pred)


print('Accuracy of classifier on test set: {:.3f}'.format(clf1.score(X2_test, y2_test)))

print("10-fold cross validation average accuracy of clf1: %.3f" % (results.mean()))

print('Confusion Matrix for Classfier:')
print(confusion_matrix_y2)

print('Classification Report for Classfier:')
print(classification_report(y2_test, y2_pred))


F1 0.6115; AUC 0.6271 
Accuracy of classifier on test set: 0.599
10-fold cross validation average accuracy of clf1: 0.542
Confusion Matrix for Classfier:
[[30 14]
 [41 52]]
Classification Report for Classfier:
             precision    recall  f1-score   support

          0       0.42      0.68      0.52        44
          1       0.79      0.56      0.65        93

avg / total       0.67      0.60      0.61       137

